## **1. Grouping and Aggregation (.groupby())**
The "group by" operation involves three steps:
1. **Splitting:** Splitting the data into groups based on some criteria.
2. **Applying:** Applying a function to each group independently.
3. **Combining:** Combining the results into a data structure.
- This is extremely powerful for summarizing data.

In [2]:
import pandas as pd

# We'll use our clean sales data from the previous module for this part.
# Let's create it again for this notebook to be self-contained.
sales_data = {
    'Transaction ID': [1001, 1002, 1003, 1004, 1005, 1006, 1007],
    'Date': ['2023-01-15', '2023-01-16', '2023-01-16', '2023-01-17', '2023-01-18', '2023-01-18', '2023-01-18'],
    'Product Category': ['Electronics', 'Office Supplies', 'Electronics', 'Home Goods', 'Office Supplies', 'Electronics', 'Home Goods'],
    'Product Name': ['Laptop', 'Pen Set', 'Mouse', 'Coffee Maker', 'Notebook', 'Laptop', 'Blender'],
    'Units Sold': [2, 10, 5, 1, 20, 1, 2],
    'Unit Price': [1200, 15, 25, 80, 5, 1250, 50]
}
sales_df = pd.DataFrame(sales_data)
sales_df['Total Revenue'] = sales_df['Units Sold'] * sales_df['Unit Price']
sales_df.head()

,Transaction ID,Date,Product Category,Product Name,Units Sold,Unit Price,Total Revenue
0,1001,2023-01-15,Electronics,Laptop,2,1200,2400
1,1002,2023-01-16,Office Supplies,Pen Set,10,15,150
2,1003,2023-01-16,Electronics,Mouse,5,25,125
3,1004,2023-01-17,Home Goods,Coffee Maker,1,80,80
4,1005,2023-01-18,Office Supplies,Notebook,20,5,100


- **Basic Grouping and Aggregation:**

In [3]:
# What is the total number of units sold per product category?
# 1. Group by 'Product Category'
# 2. Select the 'Units Sold' column
# 3. Apply the sum() aggregation

units_by_category = sales_df.groupby('Product Category')['Units Sold'].sum()

print("--- Total Units Sold by Category ---")
print(units_by_category)

# What is the average total revenue per product category?
avg_revenue_by_category = sales_df.groupby('Product Category')['Total Revenue'].mean()
print("\n--- Average Revenue by Category ---")
print(avg_revenue_by_category.round(2))

--- Total Units Sold by Category ---
Product Category
Electronics         8
Home Goods          3
Office Supplies    30
Name: Units Sold, dtype: int64

--- Average Revenue by Category ---
Product Category
Electronics        1258.33
Home Goods           90.00
Office Supplies     125.00
Name: Total Revenue, dtype: float64


- **Multiple Aggregations with .agg():**
You can apply multiple functions at once.

In [4]:
# Get the sum, mean, and count for Total Revenue, grouped by category

stats_by_category = sales_df.groupby('Product Category')['Total Revenue'].agg(['sum', 'mean', 'count'])
print("\n--- Multiple Stats by Category ---")
stats_by_category


--- Multiple Stats by Category ---


,sum,mean,count
Product Category,,,
Electronics,3775,1258.333333,3
Home Goods,180,90.000000,2
Office Supplies,250,125.000000,2


- **Grouping by Multiple Columns:**

In [5]:
# Get the total units sold, grouped by both Category and Product Name

units_by_product = sales_df.groupby(['Product Category', 'Product Name'])['Units Sold'].sum()
print("\n--- Units Sold by Category and Product Name ---")
print(units_by_product) # This results in a MultiIndex Series


--- Units Sold by Category and Product Name ---
Product Category  Product Name
Electronics       Laptop           3
                  Mouse            5
Home Goods        Blender          2
                  Coffee Maker     1
Office Supplies   Notebook        20
                  Pen Set         10
Name: Units Sold, dtype: int64


## **2. Merging, Joining, and Concatenating**
This is how you combine data from different DataFrames, similar to JOIN in SQL.

In [6]:
# Load the datasets we created for this module
employees_df = pd.read_csv("employees.csv")
departments_df = pd.read_csv("departments.csv")

print("--- Employees DataFrame ---")
employees_df

--- Employees DataFrame ---


,Employee ID,Name,Department ID
0,E001,Alice,101
1,E002,Bob,102
2,E003,Charlie,101
3,E004,David,103
4,E005,Eve,102
5,E006,Frank,101
6,E007,Grace,104


In [7]:
print("\n--- Departments DataFrame ---")
departments_df


--- Departments DataFrame ---


,Dept ID,Department Name,Location
0,101,Marketing,New York
1,102,Sales,Chicago
2,103,IT,San Francisco
3,105,Finance,New York


**pd.merge() (The most flexible method, similar to SQL JOIN):**
- **on='key_column':** The column to join on (must have the same name in both DataFrames).
- **left_on='key_in_left_df', right_on='key_in_right_df':** If key columns have different names.
- **how='inner':** (Default) Returns only the rows where the key exists in both DataFrames.
- **how='outer':** Returns all rows from both DataFrames, filling in NaN where there's no match.
- **how='left':** Returns all rows from the left DataFrame and matched rows from the right.
- **how='right':** Returns all rows from the right DataFrame and matched rows from the left.

In [8]:
# The key columns are 'Department ID' and 'Dept ID'. They have different names.
# We want to add the Department Name and Location to each employee.

# Inner Join: only includes departments present in both tables (101, 102, 103)
# Employee Grace (dept 104) and Finance dept (105) will be excluded.
inner_merged_df = pd.merge(employees_df, departments_df,
                           left_on='Department ID', right_on='Dept ID',
                           how='inner')
print("\n--- INNER Merged DataFrame ---")
inner_merged_df


--- INNER Merged DataFrame ---


,Employee ID,Name,Department ID,Dept ID,Department Name,Location
0,E001,Alice,101,101,Marketing,New York
1,E002,Bob,102,102,Sales,Chicago
2,E003,Charlie,101,101,Marketing,New York
3,E004,David,103,103,IT,San Francisco
4,E005,Eve,102,102,Sales,Chicago
5,E006,Frank,101,101,Marketing,New York


In [9]:
# Left Join: includes ALL employees, even if their department isn't in the departments table.
left_merged_df = pd.merge(employees_df, departments_df,
                          left_on='Department ID', right_on='Dept ID',
                          how='left')
print("\n--- LEFT Merged DataFrame ---")
left_merged_df # Notice Grace now has NaN for Dept Name/Location


--- LEFT Merged DataFrame ---


,Employee ID,Name,Department ID,Dept ID,Department Name,Location
0,E001,Alice,101,101.0,Marketing,New York
1,E002,Bob,102,102.0,Sales,Chicago
2,E003,Charlie,101,101.0,Marketing,New York
3,E004,David,103,103.0,IT,San Francisco
4,E005,Eve,102,102.0,Sales,Chicago
5,E006,Frank,101,101.0,Marketing,New York
6,E007,Grace,104,NaN,NaN,NaN


**pd.concat(): Stacks DataFrames on top of each other (vertically) or side-by-side (horizontally).**

In [10]:
# Let's create another small DataFrame
new_employees = pd.DataFrame([
    {'Employee ID': 'E008', 'Name': 'Heidi', 'Department ID': 102}
])

# Concatenate (stack) the new employees with the original employees
all_employees_df = pd.concat([employees_df, new_employees], ignore_index=True)
# ignore_index=True resets the index of the new DataFrame
print("\n--- Concatenated DataFrame ---")
all_employees_df


--- Concatenated DataFrame ---


,Employee ID,Name,Department ID
0,E001,Alice,101
1,E002,Bob,102
2,E003,Charlie,101
3,E004,David,103
4,E005,Eve,102
5,E006,Frank,101
6,E007,Grace,104
7,E008,Heidi,102


## **Exercises**

**1. Grouping and Aggregation:**
- Using the sales_df from the examples:
- Calculate the total Total Revenue for each Product Category.
- Find the average Unit Price for each Product Category.
- For each Product Category, find the maximum Units Sold in a single transaction and the total Total Revenue. Use the .agg() method to do this in one step.

In [25]:
sales_df

,Transaction ID,Date,Product Category,Product Name,Units Sold,Unit Price,Total Revenue
0,1001,2023-01-15,Electronics,Laptop,2,1200,2400
1,1002,2023-01-16,Office Supplies,Pen Set,10,15,150
2,1003,2023-01-16,Electronics,Mouse,5,25,125
3,1004,2023-01-17,Home Goods,Coffee Maker,1,80,80
4,1005,2023-01-18,Office Supplies,Notebook,20,5,100
5,1006,2023-01-18,Electronics,Laptop,1,1250,1250
6,1007,2023-01-18,Home Goods,Blender,2,50,100


In [26]:
total_revenue_by_category = sales_df.groupby('Product Category')['Total Revenue'].sum()

print(f"Category wise Total Revenue:\n{total_revenue_by_category}")

Category wise Total Revenue:
Product Category
Electronics        3775
Home Goods          180
Office Supplies     250
Name: Total Revenue, dtype: int64


In [27]:
avg_unit_price_by_product_category = sales_df.groupby('Product Category')['Unit Price'].mean()

print(f"Category wise Average Unit Price:\n{avg_unit_price_by_product_category}")

Category wise Average Unit Price:
Product Category
Electronics        825.0
Home Goods          65.0
Office Supplies     10.0
Name: Unit Price, dtype: float64


In [28]:
result = sales_df.groupby('Product Category').agg(
    Max_Units_Sold=('Units Sold', 'max'),
    Total_Revenue=('Total Revenue', 'sum'))

print(result)

                  Max_Units_Sold  Total_Revenue
Product Category                               
Electronics                    5           3775
Home Goods                     2            180
Office Supplies               20            250


**2. Merging DataFrames:**
- Using the employees_df and departments_df from the examples:
- Perform a right join to see which departments have no employees listed.
- Perform an outer join to see all employees and all departments, showing NaNs where matches don't exist.
-Perform an inner join again, but this time, after the merge, drop the redundant Dept ID column.

In [31]:
right_merged_df = pd.merge(employees_df, departments_df,
                           left_on ='Department ID', right_on='Dept ID',
                          how= 'right')
right_merged_df

,Employee ID,Name,Department ID,Dept ID,Department Name,Location
0,E001,Alice,101.0,101,Marketing,New York
1,E003,Charlie,101.0,101,Marketing,New York
2,E006,Frank,101.0,101,Marketing,New York
3,E002,Bob,102.0,102,Sales,Chicago
4,E005,Eve,102.0,102,Sales,Chicago
5,E004,David,103.0,103,IT,San Francisco
6,NaN,NaN,NaN,105,Finance,New York


so, Finance departments have no employees listed

In [32]:
outer_merged_df1 = pd.merge(employees_df, departments_df,
                           left_on ='Department ID', right_on='Dept ID',
                          how= 'outer')
outer_merged_df1

,Employee ID,Name,Department ID,Dept ID,Department Name,Location
0,E001,Alice,101.0,101.0,Marketing,New York
1,E003,Charlie,101.0,101.0,Marketing,New York
2,E006,Frank,101.0,101.0,Marketing,New York
3,E002,Bob,102.0,102.0,Sales,Chicago
4,E005,Eve,102.0,102.0,Sales,Chicago
5,E004,David,103.0,103.0,IT,San Francisco
6,E007,Grace,104.0,NaN,NaN,NaN
7,NaN,NaN,NaN,105.0,Finance,New York


In [33]:
inner_merged_df1 = pd.merge(employees_df, departments_df,
                           left_on ='Department ID', right_on='Dept ID',
                          how= 'inner')
inner_merged_df1

,Employee ID,Name,Department ID,Dept ID,Department Name,Location
0,E001,Alice,101,101,Marketing,New York
1,E002,Bob,102,102,Sales,Chicago
2,E003,Charlie,101,101,Marketing,New York
3,E004,David,103,103,IT,San Francisco
4,E005,Eve,102,102,Sales,Chicago
5,E006,Frank,101,101,Marketing,New York


In [34]:
inner_cleaned_df = inner_merged_df1.drop(columns= ['Dept ID'])

print("\n--- Inner Join Result (with redundant 'Dept ID' dropped) ---")
inner_cleaned_df


--- Inner Join Result (with redundant 'Dept ID' dropped) ---


,Employee ID,Name,Department ID,Department Name,Location
0,E001,Alice,101,Marketing,New York
1,E002,Bob,102,Sales,Chicago
2,E003,Charlie,101,Marketing,New York
3,E004,David,103,IT,San Francisco
4,E005,Eve,102,Sales,Chicago
5,E006,Frank,101,Marketing,New York


**3. Concatenation Challenge:**
- Create two simple DataFrames:
    - `df1 = pd.DataFrame({'A': ['A0', 'A1'], 'B': ['B0', 'B1']})`
    - `df2 = pd.DataFrame({'A': ['A2', 'A3'], 'B': ['B2', 'B3']})`
- Concatenate them vertically (stacking df2 below df1).
- Now, create two different DataFrames:
    - `df3 = pd.DataFrame({'A': ['A0', 'A1'], 'B': ['B0', 'B1']})`
    - `df4 = pd.DataFrame({'C': ['C0', 'C1'], 'D': ['D0', 'D1']})`
- Concatenate them horizontally (placing df4 to the right of df3). Hint: you'll need to use axis=1.

In [35]:
df1 = pd.DataFrame({'A': ['A0', 'A1'], 'B': ['B0', 'B1']})
df2 = pd.DataFrame({'A': ['A2', 'A3'], 'B': ['B2', 'B3']})

In [39]:
vertical_df = pd.concat([df1,df2], ignore_index = True)
print("Vertically stacking:")
vertical_df

Vertically stacking:


,A,B
0,A0,B0
1,A1,B1
2,A2,B2
3,A3,B3


In [41]:
df3 = pd.DataFrame({'A': ['A0', 'A1'], 'B': ['B0', 'B1']})
df4 = pd.DataFrame({'C': ['C0', 'C1'], 'D': ['D0', 'D1']})

horizontal_df = pd.concat([df3,df4], axis=1)
print("Horizontally stacking:")
horizontal_df

Horizontally stacking:


,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
